In [22]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os


In [23]:
## Load and preprocess new data
df = pd.read_csv('Fill.csv')
x = np.array(df.iloc[:, :2])
y = np.array(df.iloc[:, 2])

scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)

scaler_y = StandardScaler()
y = y.reshape(-1, 1)
y = scaler_y.fit_transform(y)

x = torch.from_numpy(x).type(torch.FloatTensor)
y = torch.from_numpy(y).type(torch.FloatTensor)


In [24]:
# Define the RNN model
class Net(nn.Module):
    def __init__(self, input_size, hidden_dim, num_layers, output=1):
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.linear(out)
        return out



In [25]:
# Load the model
model_path = './rnnmodel/model_rnn_weight.pth'
model = Net(hidden_dim=220, input_size=1, num_layers=2)
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Model loaded from disk.")
else:
    print("Model file not found.")
    

Model loaded from disk.


In [26]:
# Load the model
model_path = './rnnmodel/model_rnn_weight.pth'
model = Net(hidden_dim=220, input_size=1, num_layers=2)
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Model loaded from disk.")
else:
    print("Model file not found.")

# Prepare data for testing
x = x.reshape(-1, 2, 1)

# Testing the model
model.eval()
with torch.no_grad():
    outputs = model(x)

# Inverse transform
outputs = outputs.numpy()
y = y.numpy()
outputs = scaler_y.inverse_transform(outputs)
y = scaler_y.inverse_transform(y)

# Calculate Metrics
mse = mean_squared_error(y, outputs)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, outputs)
r2 = r2_score(y, outputs)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

Model loaded from disk.
Mean Squared Error (MSE): 272.9139709472656
Root Mean Squared Error (RMSE): 16.52010726928711
Mean Absolute Error (MAE): 13.262194633483887
R² Score: -0.5788819485610244
